In [26]:
class Test:
    # Set up all the constants of the system, these are chosen after optimizing for
    # performance and accuracy
    # Sample set to default - 1.25 GSa/sec
    sample_rate = 0.625e9
    # Synchronize analog and digital channels
    digital_sync_length = 9e-9
    # Null pulse to settle instruments
    null_pulse_length = 30e-9
    # MW trigger length, detected at POS edge
    mw_trig_length = 30e-9
    # Max samples supported by AWG
    max_samples = 130e6
    
    laser_readout_length = 250e-9
    delay_length = 1e-6
    pi_pulse_length = 100e-9
    freq_rep = 10000
    
    def list_to_waveform_pulsed(self):
        # Laser, Readout, Switch and MW trigger channels
        analog_samples = {'a_ch0': []}
        digital_samples = {'d_ch0': [], 'd_ch1': [], 'd_ch2': [], 'd_ch5': []}

        # Length of a single frequency (single pulse * frequency repetition rate)
        single_freq_pulse_length = (self.laser_readout_length + self.delay_length + self.pi_pulse_length + 
                                    self.null_pulse_length) * self.freq_rep
        single_freq_pulse_samples = int(np.floor(single_freq_pulse_length * self.sample_rate))

        total_pulse_length = self.mw_trig_length + 2 * self.null_pulse_length + single_freq_pulse_length
        total_pulse_samples = int(np.floor(total_pulse_length * self.sample_rate))

        # Set up all the pulse lengths
        mw_trig_pulse_sample = int(np.floor(self.mw_trig_length * self.sample_rate))
        delay_pulse_sample = int(np.floor(self.delay_length * self.sample_rate))
        switch_pulse_sample = int(np.floor(self.pi_pulse_length * self.sample_rate))
        null_pulse_sample = int(np.floor(self.null_pulse_length * self.sample_rate))
        laser_readout_pulse_sample = int(np.floor(self.laser_readout_length * self.sample_rate))

        # Set up empty sequences for channels (switch uses np.ones as channel HIGH is off and LOW is on)
        mw_trig_samples = np.zeros(total_pulse_samples)
        laser_samples = np.zeros(total_pulse_samples)
        readout_samples = np.zeros(total_pulse_samples)
        switch_samples = np.ones(total_pulse_samples)

        mw_trig_samples[0:mw_trig_pulse_sample] = 1

        current_freq_idx = mw_trig_pulse_sample + 2 * null_pulse_sample
        
        for _ in range(self.freq_rep):
            # Null pulses are added to take into account the settling time of the instruments
            single_freq_start = current_freq_idx

            laser_start = single_freq_start
            laser_stop = laser_start + laser_readout_pulse_sample

            switch_start = single_freq_start + delay_pulse_sample
            switch_stop = switch_start + switch_pulse_sample

            laser_samples[laser_start:laser_stop] = np.ones(laser_stop - laser_start)
            readout_samples[laser_start:laser_stop] = np.ones(laser_stop - laser_start)
            switch_samples[switch_start:switch_stop] = np.zeros(switch_stop - switch_start)

            current_freq_idx += laser_readout_pulse_sample + delay_pulse_sample + switch_pulse_sample + \
                                null_pulse_sample

            # Digital and analog channels are mapped to the sequences
            digital_samples['d_ch0'] = laser_samples
            digital_samples['d_ch1'] = readout_samples
            digital_samples['d_ch2'] = switch_samples
            digital_samples['d_ch5'] = mw_trig_samples
            analog_samples['a_ch0'] = np.zeros(total_pulse_samples)

        return analog_samples, digital_samples

In [27]:
t = Test()
analog_samples, digital_samples = t.list_to_waveform_pulsed()

In [28]:
for idx, (key, value) in enumerate(analog_samples.items()):
    print(len(value))

8625056


In [21]:
fig, ax = plt.subplots(nrows=len(digital_samples), sharex=True)

for idx, (key, value) in enumerate(digital_samples.items()):
    ax[idx].plot(value)
    ax[idx].set_title(key)
    print(len(value))
    
fig.tight_layout()

plt.savefig("digital.png", dpi=300)

4368
4368
4368
4368


In [ ]:
single_sweep_time = 2
total_pulse_length = 0.014
freq_list_len = 41
freq_rep = 10000

In [ ]:
avg